# Previous Work

The previous net saved in _'01-07-17_convnet.h5'_ , gets a 97% accuracy rate on the test dataset (!) of GTSRB. However, when it must predict on images that don't belong to the GTSRB dataset, it sees traffic signs where there are none, and with very high (~100%) confidence. This is a problem because during a race, the car won't see any traffic signs most of the time.

The approach to fix this, is to add a _background_ class, also called _Zero_ class, as the 43th class to the dataset.

In [ ]:
from gtsrb_loader.load_data import load_data
from gtsrb_loader.get_folderpath import get_folderpath
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
path = get_folderpath(subset='train', original_images=False)
X_train, y_train = load_data(path)

In [ ]:
path_zero = "C:\\Users\\tomas\\Desktop\\GTSRB\\Zero_Class\\ 00043"
X_train_zero, y_train_zero = load_data(path_zero)

In [ ]:
from skimage.transform import resize
X_train_zero = np.array([resize(pic, (64, 64), mode='edge') for pic in X_train_zero])

# Training Set Selection

For this second net, we want to distinguish between _background_ and _traffic sign_ reliably. Thats why I dont balance number of training images per class yet. Any traffic sign does the job. Since our zero class is only in gray color, we will transfor the GTSRB data to grayscale

# Black and White Zero class

In [ ]:
from skimage import color
x = color.rgb2gray(X_train[0])
x.shape

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(X_train[0])
plt.subplot(1, 2, 2)
plt.imshow(x, cmap='gray')
plt.show()

In [ ]:
X_train_gray = np.array([color.rgb2gray(pic) for pic in X_train])

In [ ]:
X_train_zero.shape, X_train_gray.shape

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
X_train_zero = X_train_zero.reshape(len(X_train_zero), 64, 64, 1) # grayscale wrapper for the datagen object 
X_train_zero.shape

In [ ]:
datagen.fit(X_train_zero)

In [ ]:
X_train_gray.max(), X_train_gray.min()

# Color zero class
since we have a colorfull zeroclass we do not need to transform the pictures to grey


In [ ]:
#path = get_folderpath(subset='train', original_images=False)
path="/data_on_server/Images/GTSRB_64x64/Final_Test"
X_train, y_train = load_data(path)


In [ ]:
#path_zero = "Y:\\Zero_Class_color\\00043"
path_zero = "//data_on_server//Zero_Class_color"
X_train_zero, y_train_zero = load_data(path_zero)

In [ ]:
X_trainC=X_train+X_train_zero
Y_trainC=y_train+y_train_zero

In [ ]:
sign=np.array(Y_trainC)
unique_classes = len(np.unique(sign))
y_trainN = np.eye(unique_classes)[np.array(sign, dtype=int)]
X_trainN = np.array(X_trainC, dtype=np.float32) / 255

In [ ]:
for n in X_trainN:
    if(n.shape != (64,64,3)):
        print(n.shape)

In [ ]:
np.array(y_trainN).shape

In [ ]:
X_train = X_trainN.reshape(len(X_train), 64, 64, 3) # grayscale wrapper for the datagen object 
X_train.shape

# CHECKPOINT WIP

In [ ]:
# normalize X_train
X_train = np.array(X_train, dtype=np.float32) / 255
# X_train_zero

In [ ]:
# one-hot encode the labels
boundBox=np.array(y_train)[:,0:4]
sign=np.array(y_train)[:,4]
unique_classes = len(np.unique(sign))
y_trainN = np.eye(unique_classes)[np.array(sign, dtype=int)]
y_trainF=np.concatenate([y_trainN,boundBox], axis=1)

In [ ]:
y_trainF



## From Keras docs, VGG-like Convnet.

First Block

In [ ]:
# Keras Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(input_shape=(64, 64, 3),
                 filters=32,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(Conv2D(filters=32,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Dropout(0.25))

Second Block

In [ ]:
model.add(Conv2D(filters=64,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(Conv2D(filters=64,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Dropout(0.25))

Fully Connected Layers

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(44, activation='softmax'))

In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
# define model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_data=(X_test, Y_test),
          shuffle=True,
          callbacks=[tensorboard])

In [ ]:
model.fit(X_trainN, y_trainN, batch_size=32, verbose=2 ,epochs=20)

In [ ]:
model.save('16-10-17_convnet_Zero_color_E20.h5')